In [1]:
# %load_ext autoreload
# %autoreload
# %matplotlib inline

from news_crawler import NewsCrawler
import glob
import json
from IPython.core.debugger import set_trace
from pathlib import Path
from pprint import pprint
import spacy
from tokenizer import MergedTokenizer
import time

import gensim
import gensim.corpora as corpora

import pyLDAvis
import pyLDAvis.gensim as gensimvis

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

from tqdm.auto import tqdm
tqdm.pandas()

C:\Users\infomax\Anaconda3\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
nlp = spacy.load('en_core_web_lg')

In [4]:
fnames = glob.glob('newsdata/downloaded/*.json')[:50]

In [4]:
merged_tokenizer = MergedTokenizer(nlp, merge_chunks=False, merge_ner=True)

In [5]:
data = []
for fname in tqdm(fnames):
    js = json.loads(Path(fname).read_text())
    doc = merged_tokenizer.lemmatized(js['text'])
    data.append(doc)

In [6]:
id2word = corpora.Dictionary(data)

In [7]:
corpus = [id2word.doc2bow(text) for text in data]

In [8]:
s = time.time()
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, 
                                            id2word=id2word, 
                                            num_topics=5, 
                                            random_state=100, 
                                            update_every=None, 
                                            chunksize=1, 
                                            passes=5, 
                                            alpha='auto', 
                                            per_word_topics=True)
pprint(time.time()-s)

8.586445569992065


In [11]:
lda_model = gensim.models.HdpModel(corpus=corpus, id2word=id2word)

In [12]:
lda_model.show_topics()

[(0,
  '0.012*say + 0.004*new + 0.004*time + 0.004*people + 0.003*trump + 0.003*president + 0.002*state + 0.002*company + 0.002*getty + 0.002*world + 0.002*come + 0.002*image + 0.002*tell + 0.002*go + 0.002*work + 0.002*good + 0.002*know + 0.002*report + 0.002*include + 0.002*high'),
 (1,
  '0.012*say + 0.004*trump + 0.004*time + 0.004*new + 0.003*people + 0.003*president + 0.003*good + 0.003*state + 0.003*go + 0.003*tell + 0.003*work + 0.002*come + 0.002*know + 0.002*want + 0.002*take + 0.002*house + 0.002*report + 0.002*think + 0.002*company + 0.002*world'),
 (2,
  '0.009*say + 0.003*time + 0.002*people + 0.002*new + 0.002*company + 0.002*work + 0.002*know + 0.002*go + 0.002*good + 0.002*start + 0.002*outstanding + 0.002*state + 0.001*game + 0.001*product + 0.001*come + 0.001*change + 0.001*shampoo + 0.001*tell + 0.001*want + 0.001*add'),
 (3,
  '0.007*say + 0.006*trump + 0.003*people + 0.003*president + 0.002*getty + 0.002*time + 0.002*new + 0.002*think + 0.002*chinese + 0.002*come 

In [17]:
pyLDAvis.enable_notebook()

In [ ]:
'''
여기를 실행시키려면 반드시 future 패키지를 0.18로 업그레이드한다
pip install future --upgrade
'''

vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

In [62]:
import glob
import json
import os
import re
import operator
import matplotlib.pyplot as plt
import warnings
import gensim
import numpy as np
# warnings.filterwarnings('ignore')

import nltk
nltk.download('stopwords')
nltk.download('wordnet')

from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary
from pprint import pprint
from smart_open import smart_open
from pathlib import Path

from tqdm.auto import tqdm
tqdm.pandas()

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\infomax\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\infomax\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [46]:
def build_texts(date):
    fnames = glob.glob('newsdata/downloaded/*.json')
    for fname in tqdm(fnames[:50000]):
        js = json.loads(Path(fname).read_text())
        if js['published_at']==date:
            yield gensim.utils.simple_preprocess(js['text'], deacc=True, min_len=3)

In [47]:
train_texts = list(build_texts('2019-10-13'))

In [48]:
len(train_texts)

1045

In [49]:
bigram = gensim.models.Phrases(train_texts)

In [50]:
from gensim.utils import lemmatize
from nltk.corpus import stopwords

In [51]:
stops = set(stopwords.words('english'))

In [52]:
def process_texts(texts):
    texts = [[word for word in line if word not in stops] for line in texts]
    texts = [bigram[line] for line in texts]
    
    from nltk.stem.wordnet import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    
    #texts = [[word for word in lemmatizer.lemmatize(' '.join(line), pos='v').split()] for line in texts]
    texts = [[lemmatizer.lemmatize(word, pos='v') for word in line] for line in texts]
    return texts

In [53]:
texts = process_texts(train_texts)

In [54]:
texts[0]

['surprise',
 'one',
 'donald_trump',
 'factor',
 'heavily',
 'hate',
 'six',
 'part',
 'discovery',
 'channel',
 'documentary',
 'series',
 'executive',
 'produce',
 'steven',
 'spielberg',
 'alex',
 'gibney',
 'investigate',
 'many',
 'form',
 'cause',
 'possible',
 'remedy',
 'intolerance',
 'ubiquitous',
 'presence',
 'fact',
 'even',
 'specific',
 'topic',
 'conversation',
 'corrosive',
 'shadow',
 'loom',
 'large',
 'proceed',
 'international',
 'criminal',
 'lawyer',
 'patricia',
 'viseur',
 'sellers',
 'describe',
 'khmer',
 'rouge',
 'madman',
 'pol',
 'pot',
 'motivations',
 'initiate',
 'genocide',
 'state',
 'pause',
 'indicate',
 'know',
 'imply',
 'go',
 'make',
 'cambodia',
 'great',
 'current',
 'president',
 'ugly',
 'divisive',
 'language',
 'policies',
 'white',
 'nationalist',
 'movements',
 'mass',
 'shoot',
 'inspire',
 'natural',
 'fit',
 'directors',
 'geeta',
 'gandbhir',
 'sam',
 'pollard',
 'non',
 'fiction',
 'work',
 'premier',
 'oct',
 'trump',
 'perfectly

In [66]:
dictionary = Dictionary(texts)
dictionary.filter_extremes(no_below=20, no_above=0.5)
dictionary.compactify()

In [67]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [68]:
ldamodel = LdaModel(corpus=corpus, 
                    num_topics=20, 
                    passes=20, 
                    iterations=400,
                    id2word=dictionary, 
                    chunksize=2000, 
                    alpha='auto', 
                    eta='auto',
                    eval_every=None,
                    per_word_topics=True)

In [69]:
ldamodel.show_topics(formatted=False)

[(1,
  [('group', 0.015083032),
   ('use', 0.013081429),
   ('charge', 0.011085095),
   ('action', 0.009851375),
   ('report', 0.00952622),
   ('include', 0.008773945),
   ('bill', 0.008577762),
   ('law', 0.008533424),
   ('government', 0.0072554313),
   ('muslim', 0.007079259)]),
 (15,
  [('september', 0.021326797),
   ('october', 0.014441494),
   ('london', 0.0137893045),
   ('women', 0.0102636665),
   ('plastic', 0.009176267),
   ('afp_getty', 0.008313754),
   ('british', 0.00824436),
   ('climate_change', 0.007137565),
   ('university', 0.007125892),
   ('club', 0.0070082955)]),
 (11,
  [('time', 0.01979444),
   ('eat', 0.014320884),
   ('dance', 0.010208243),
   ('ball', 0.009190754),
   ('gate', 0.009028563),
   ('day', 0.0085413),
   ('show', 0.008279967),
   ('food', 0.008145665),
   ('would', 0.0074766385),
   ('get', 0.0072804857)]),
 (3,
  [('afp_getty', 0.013029872),
   ('tokyo', 0.012728204),
   ('people', 0.012454457),
   ('flood', 0.010953641),
   ('typhoon_hagibis', 0.

In [71]:
ldamodel.top_topics(corpus)

[([(0.01676476, 'trump'),
   (0.010628187, 'president'),
   (0.008923994, 'think'),
   (0.0084948065, 'know'),
   (0.007867411, 'want'),
   (0.007176695, 'go'),
   (0.00689303, 'right'),
   (0.006655473, 'get'),
   (0.0066405595, 'would'),
   (0.006212451, 'tell'),
   (0.006028704, 'call'),
   (0.005881559, 'well'),
   (0.0057155266, 'like'),
   (0.00569217, 'need'),
   (0.0052375253, 'time'),
   (0.004426728, 'see'),
   (0.0041395375, 'way'),
   (0.0041143587, 'work'),
   (0.0040757675, 'come'),
   (0.004060522, 'even')],
  -0.7901351186404558),
 ([(0.026192456, 'turkey'),
   (0.025372317, 'syria'),
   (0.018896881, 'force'),
   (0.013141196, 'fight'),
   (0.012501256, 'sunday'),
   (0.01232539, 'camp'),
   (0.01200878, 'border'),
   (0.010761353, 'islamic_state'),
   (0.009730615, 'turkish'),
   (0.009448389, 'troop'),
   (0.008655202, 'isis'),
   (0.008491832, 'northern_syria'),
   (0.0077964473, 'kurdish_led'),
   (0.007744727, 'ally'),
   (0.007641925, 'trump'),
   (0.0075268303, 

In [59]:
lsimodel = LsiModel(corpus=corpus, num_topics=10, id2word=dictionary)

In [36]:
lsimodel.show_topics(num_topics=5, formatted=False)

[(0,
  [('noon', -0.6613216968723826),
   ('lane', -0.46454260670204767),
   ('ave', -0.2841907711589004),
   ('way', -0.2583361514239686),
   ('lane_noon', -0.13663017844177874),
   ('hill', -0.1096466067462519),
   ('ave_noon', -0.09837372847808079),
   ('way_noon', -0.09837372847808079),
   ('park', -0.09349149630332552),
   ('county', -0.08767795208114522)]),
 (1,
  [('say', 0.49071339033150546),
   ('one', 0.16044114629089629),
   ('also', 0.1437593795008963),
   ('jesse', 0.1379700929949128),
   ('make', 0.13474691104042738),
   ('get', 0.11741075749882347),
   ('would', 0.11709819554865286),
   ('china', 0.11259388004910137),
   ('take', 0.10266960571547686),
   ('people', 0.10163539160551438)]),
 (2,
  [('jesse', -0.680802036078172),
   ('walt', -0.38078757950135045),
   ('say', 0.24994723380790815),
   ('gus', -0.11539017560646977),
   ('get', -0.10549742394335948),
   ('meth', -0.10385115804582276),
   ('breaking_bad', -0.09231214048517587),
   ('drug', -0.09191937442831924),

In [16]:
hdpmodel = HdpModel(corpus=corpus, id2word=dictionary, T=20)

In [17]:
hdpmodel.show_topics(formatted=False)

[(0,
  [('would', 0.004295570439678243),
   ('get', 0.004084392221812674),
   ('time', 0.0031267363718436194),
   ('take', 0.0031216896466816856),
   ('trump', 0.003086331857143635),
   ('people', 0.002989355167769212),
   ('go', 0.0027683690865242306),
   ('first', 0.0026692515921386325),
   ('come', 0.002633617384499611),
   ('back', 0.002588141019661147),
   ('two', 0.0025636002967385336),
   ('leave', 0.002489021551703571),
   ('could', 0.0024684194742144025),
   ('tell', 0.0024155139028360305),
   ('like', 0.0024112782699790207),
   ('new', 0.002320258854222873),
   ('work', 0.002264318715619079),
   ('sunday', 0.0021850215213367417),
   ('start', 0.0021762315781064055),
   ('call', 0.0021607537668540317)]),
 (1,
  [('syria', 0.0032018558555601487),
   ('turkey', 0.0026285795119861846),
   ('trump', 0.0025475580610988735),
   ('force', 0.0022063303996782903),
   ('would', 0.0021094460737535363),
   ('report', 0.0018491401382663712),
   ('northern_syria', 0.0018027722149259193),
  

In [33]:
def ret_top_model():
    """
    Since LDAmodel is a probabilistic model, it comes up different topics each time we run it. To control the
    quality of the topic model we produce, we can see what the interpretability of the best topic is and keep
    evaluating the topic model until this threshold is crossed. 
    
    Returns:
    -------
    lm: Final evaluated topic model
    top_topics: ranked topics in decreasing order. List of tuples
    """
    top_topics = [(0, 0)]
    while top_topics[0][1] < 0.9:
        lm = LdaModel(corpus=corpus, id2word=dictionary, eval_every=None, num_topics=100, passes=20, alpha='auto', eta='auto', per_word_topics=True, chunksize=2000)
        coherence_values = {}
        for n, topic in lm.show_topics(num_topics=-1, formatted=False):
            topic = [word for word, _ in topic]
            cm = CoherenceModel(topics=[topic], texts=texts, dictionary=dictionary, window_size=10)
            coherence_values[n] = cm.get_coherence()
        top_topics = sorted(coherence_values.items(), key=operator.itemgetter(1), reverse=True)
        print(top_topics[0][1])
    return lm, top_topics

In [34]:
lm, top_topics = ret_top_model()

0.8101233016556432
0.7530478649782473
0.6938712938987388
0.7496064271145078
0.7341531118233621
0.7530478649782473
0.7068196405329843
0.8055868758230924
0.7530478649782473
0.7590263732732534
0.7530478649782473


KeyboardInterrupt: 

In [18]:
import pyLDAvis.gensim

In [19]:
pyLDAvis.enable_notebook()

In [20]:
# %%prun
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

C:\Users\infomax\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [21]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.008799  0.040010       1        1  28.058151
3     -0.088888  0.080434       2        1  13.211243
8     -0.155845 -0.118758       3        1  11.006627
4     -0.032828  0.092712       4        1   9.822680
2     -0.005963  0.007087       5        1   8.331971
6      0.085059 -0.008101       6        1   7.085956
1      0.042019 -0.126956       7        1   6.817401
5      0.017827  0.001181       8        1   6.059510
0      0.025799  0.061129       9        1   5.496841
9      0.121618 -0.028736      10        1   4.109619, topic_info=     Category        Freq              Term       Total  loglift  logprob
2792  Default  110.000000  margaret_brennan  110.000000  30.0000  30.0000
903   Default   92.000000            turkey   92.000000  29.0000  29.0000
897   Default   94.000000             syria   94.000000  28.0000  28.0000
2468  Default   95.000000             japan   95.000000  27.0000  27.0000
338   Default  163.000000             trump  163.000000  26.0000  26.0000
...       ...         ...               ...         ...      ...      ...
593   Topic10    6.197333               two  169.780655  -0.1185  -6.1488
555   Topic10    5.993699             since  101.035507   0.3671  -6.1822
191   Topic10    6.216462              make  225.261505  -0.3982  -6.1457
119   Topic10    6.062891             first  159.790375  -0.0798  -6.1707
332   Topic10    5.861533              time  191.159714  -0.2929  -6.2045

[694 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
8992      9  0.839453        abbie
9543      2  0.963530         abiy
2138      5  0.713529   accomplish
3693      1  0.057572    activists
3693      3  0.115144    activists
...     ...       ...          ...
9786      5  0.924965      yoshida
6148      7  0.188410  yovanovitch
6148     10  0.753642  yovanovitch
4926      3  0.947174          ypg
5138      9  0.943516       zverev

[1729 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 4, 9, 5, 3, 7, 2, 6, 1, 10])

In [72]:
from gensim.test.utils import common_corpus

In [74]:
lda = LdaModel(common_corpus, num_topics=10)

In [84]:
dictionary.id2token

{0: 'alex',
 1: 'along',
 2: 'american',
 3: 'among',
 4: 'anyone',
 5: 'area',
 6: 'aside',
 7: 'ask',
 8: 'average',
 9: 'bear',
 10: 'begin',
 11: 'belong',
 12: 'ben',
 13: 'bill',
 14: 'british',
 15: 'capacity',
 16: 'carry',
 17: 'cast',
 18: 'cause',
 19: 'central',
 20: 'century',
 21: 'certain',
 22: 'challenge',
 23: 'channel',
 24: 'church',
 25: 'circumstances',
 26: 'clear',
 27: 'comment',
 28: 'commit',
 29: 'conflict',
 30: 'conversation',
 31: 'cost',
 32: 'criminal',
 33: 'current',
 34: 'describe',
 35: 'detail',
 36: 'direct',
 37: 'discuss',
 38: 'donald_trump',
 39: 'encourage',
 40: 'especially',
 41: 'establish',
 42: 'even',
 43: 'evidence',
 44: 'executive',
 45: 'eye',
 46: 'fact',
 47: 'factor',
 48: 'fail',
 49: 'familiar',
 50: 'famous',
 51: 'far',
 52: 'feel',
 53: 'fellow',
 54: 'figure',
 55: 'first',
 56: 'fit',
 57: 'form',
 58: 'fuel',
 59: 'fundamental',
 60: 'game',
 61: 'germany',
 62: 'go',
 63: 'great',
 64: 'grind',
 65: 'group',
 66: 'hand',